In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
class Set_layer:
    def __init__(self, Length, Thermal_conductivity, Volumetric_heat_capacity, Divide):
        self.Length = Length # 레이어 길이 
        self.Cell_length = Length/Divide # 셀 길이
        self.Thermal_conductivity = Thermal_conductivity # 열전도율
        self.Volumetric_heat_capacity = Volumetric_heat_capacity # 체적열용량
        self.Divide = Divide # 차분 수
        self.Thermal_resistance = (Length/Divide)/Thermal_conductivity # 열저항
        self.Thermal_conductance = Thermal_conductivity/(Length/Divide) # 
        self.Thermal_diffusivity = Thermal_conductivity/(Volumetric_heat_capacity)

    def L(self): 
        return self.Length
    def dx(self): 
        return self.Cell_length
    def k(self): 
        return self.Thermal_conductivity
    def C(self): 
        return self.Volumetric_heat_capacity
    def n(self): 
        return self.Divide
    def R(self):
        return self.Thermal_resistance
    def K(self):
        return self.Thermal_conductance
    def alpha(self):
        return self.Thermal_diffusivity
    
# Construction 1 = [[Layer 1],[Layer 2],[Layer 3]]

def NumberOfCellsConstruction(Construction): 
    return sum([Construction[i].n() for i in range(len(Construction))])

def NumberOfCellsLayer(layer): 
    return layer.n()

def ResistanceOfConstruction(Construction): # Thermal resistance of Layer
    return sum([Construction[i].R()*NumberOfCellsLayer(Construction[i]) for i in range(len(Construction))])

def ConductanceOfConstruction(Construction): # Thermal conductance of Layer
    return 1/ResistanceOfConstruction(Construction)

# unit change
def D2K(Temperature): # Degree to Kelvin
    return Temperature + 273.15
def K2D(Temperature): # Kelvin to Degree
    return Temperature - 273.15
def None_matrix(rows, cols):
    return [[None]*cols for _ in range(rows)]  
def None_list(N):
    return [None]*N
def cm2in(value):
    return value/2.54
# unit change 
day_to_hour = 24
hour_to_min = 60
hour_to_sec = 3600
min_to_sec = 60  
m2cm = 100

# set timestep, duration
t_step = 10 # [s] 
pre_simulation_time = 60 # [h]
total_simulation_time = 168 # [h]
simulation_time = total_simulation_time - pre_simulation_time # [h]
t_row = int(total_simulation_time*hour_to_sec/t_step) # 시간 행
t_list = [t_step*i for i in range(t_row)] # 시간 행 리스트
time = pd.DataFrame(t_list)      #second
time_min = time/60;         #minute
time_hour = time/3600;      #hour
time_end = time_hour[-1:];    # last time

StartHour = 120
EndHour   = 144
resize_y0 = int(StartHour*hour_to_sec/t_step)
resize_y1 = int(EndHour*hour_to_sec/t_step)


#
T_init = D2K(20) 
T_BC1 = [(D2K(20 + 10*(math.sin(2*math.pi*n/(day_to_hour*hour_to_sec))))) for n in t_list] # [K]
T_BC2 = [(D2K(20)) for _ in t_list]

BC_L = T_BC1
BC_R = T_BC2
T0 = T_BC1

# k, C, alpha(0.25~5)x10^(-6)
alpha_unit = 10**(-6)
min_alpha = 0.25
max_alpha = 5
alpha_interval = 0.25
alpha = np.arange(min_alpha*alpha_unit, max_alpha*alpha_unit+alpha_interval*alpha_unit, alpha_interval*alpha_unit)

# Case A 
VHC = 10**6 # Volumetric Heat Cap      
TC = [alpha[i]*VHC for i in range(len(alpha))] # Thermal Conductivity
CaseName = "DF_Error_rate_Case_A.csv"

# Case B
# k = 1
# VHC = [k/alpha[i] for i in range(len(alpha))]
# CaseName = "DF_Error_rate_Case_B.csv"


min_length = 0.05
max_length = 0.5
length_interval = 0.05
length = np.arange(max_length,min_length-length_interval, -length_interval)

divide_num = np.arange(min_length*m2cm, max_length*m2cm+length_interval*m2cm, length_interval*m2cm)

# Save dataframe
x = length
y = alpha
x_pos1, y_pos1 = np.meshgrid(x, y)

In [ ]:
DF_Error_rate = pd.DataFrame(np.zeros((len(alpha),len(length))))
for alpha_idx in range(len(alpha)):
    for len_idx in range(len(length)):
        
        layer1 = Set_layer(length[len_idx], TC[alpha_idx], VHC, int(divide_num[len_idx]))    
        Wall = [layer1] 
        N = CN_of_Wall(Wall)
        dx = [Lidx.dx() for Lidx in Wall for _ in range(CN_of_layer(Lidx))]
        dx_L, dx_R = [(dx[i-1] + dx[i])/2 for i in range(1,N)], [(dx[i] + dx[i+1])/2 for i in range(N-1)]
        dx_L.insert(0,dx[0]/2)
        dx_R.append(dx[N-1]/2)
        k =    [Lidx.k() for Lidx in Wall for _ in range(CN_of_layer(Lidx))]
        C =    [Lidx.C() for Lidx in Wall for _ in range(CN_of_layer(Lidx))]
        R =    [Lidx.R() for Lidx in Wall for _ in range(CN_of_layer(Lidx))]
        K = [Lidx.K() for Lidx in Wall for _ in range(CN_of_layer(Lidx))]
        K_L, K_R = [K[i] for i in range(1,N)], [K[i] for i in range(N-1)]
        K_L.insert(0,K[0]*2)
        K_R.append(K[N-1]*2)
        rows, cols = t_row, N

        T_L, T, T_R =                        None_matrix(rows,cols), None_matrix(rows,cols), None_matrix(rows,cols)
        q_in, q, q_out =                     None_matrix(rows,cols), None_matrix(rows,cols), None_matrix(rows,cols)
        T_L_half, T_half, T_R_half =         None_matrix(rows-1,cols), None_matrix(rows-1,cols), None_matrix(rows-1,cols)
        Carneff =                            None_matrix(rows-1,cols)
        T0_half =                            None_list(rows-1)
        q_in_half, q_half, q_out_half =      None_matrix(rows-1,cols), None_matrix(rows-1,cols), None_matrix(rows-1,cols)
        U_ex_in_cell, U_ex_c_cell, U_ex_st_cell, U_ex_out_cell, U_stored_ex_cell = None_matrix(rows-1,cols), None_matrix(rows-1,cols), None_matrix(rows-1,cols), None_matrix(rows-1,cols), None_matrix(rows-1,cols)
        D_ex_in, D_ex_c, D_ex_out =          None_list(rows-1), None_list(rows-1), None_list(rows-1)

        for i in range(N):
            T_L[0][i], T[0][i], T_R[0][i] = T_init, T_init, T_init

        for n in range(t_row):
            T_L[n][0], T_R[n][N-1] = BC_L[n], BC_R[n]


        a_list = [-t_step*K_L[i] for i in range(N)]
        b_list = [2*dx[i]*C[i]+t_step*K_L[i]+t_step*K_R[i] for i in range(N)]
        c_list = [-t_step*K_R[i] for i in range(N)]

        A = np.zeros((N, N))

        for idx in range(N-1):
            A[idx+1][idx] = a_list[idx+1]
            A[idx][idx]   = b_list[idx]
            A[N-1][N-1]   = b_list[N-1]
            A[idx][idx+1] = c_list[idx]
            
        A_inv = np.linalg.inv(A)

        for n in range(t_row-1): # 타임스탭 for 문 
            B = [] 
            for i in range(1,N-1): # node for 문
                B.append([t_step*K_L[i]*T[n][i-1]+(2*dx[i]*C[i]-t_step*K_L[i]-t_step*K_R[i])*T[n][i]+t_step*K_R[i]*T[n][i+1]]) # g_i 추가 열벡터로 표현하기 위해 [] 를 씌워줌
            B.insert(0,[2*t_step*K_L[0]*T_L[n][0]     
                        + (2*dx[0]*C[0]-t_step*K_L[0]-t_step*K_R[0])*T[n][0] 
                        + t_step*K_R[0]*T[n][1]])  
            B.append([t_step*K_L[N-1]*T[n][N-2]    
                        + (2*dx[N-1]*C[N-1]-t_step*K_L[N-1]-t_step*K_R[N-1])*T[n][N-1] 
                        + 2*t_step*K_R[N-1]*T_R[n][N-1]]) 
            
            for i in range(N): 
                T[n+1][i] = np.dot(A_inv, B)[i][0]
            for i in range(1,N):
                T_L[n+1][i] = (T[n+1][i-1]+T[n+1][i])/2
            for i in range(0,N-1):
                T_R[n+1][i] = (T[n+1][i]+T[n+1][i+1])/2

        for n in range(t_row):
            for i in range(1,N):
                q_in[n][i] = K_L[i]*(T[n][i-1] - T[n][i])
            for i in range(N-1):
                q_out[n][i] = K_R[i]*(T[n][i] - T[n][i+1])
            q_in[n][0], q_out[n][N-1] = K_L[0]*(T_L[n][0]-T[n][0]), K_R[N-1]*(T[n][N-1]-T_R[n][N-1])
            for i in range(N):
                q[n][i] = (q_in[n][i] + q_out[n][i])/2

        for n in range(t_row-1):
            for i in range(N):
                q_in_half[n][i], q_half[n][i], q_out_half[n][i] = (q_in[n][i] + q_in[n+1][i])/2, (q[n][i] + q[n+1][i])/2, (q_out[n][i] + q_out[n+1][i])/2
                T_L_half[n][i], T_half[n][i], T_R_half[n][i] = (T_L[n][i] + T_L[n+1][i])/2, (T[n][i] + T[n+1][i])/2, (T_R[n][i] + T_R[n+1][i])/2
                T0_half[n] = (T0[n]+T0[n+1])/2  


        for n in range(t_row-1):
            for i in range(N):
                U_ex_in_cell[n][i] = q_in_half[n][i]*(1-(T0_half[n]/T_L_half[n][i]))
                U_ex_c_cell[n][i] = (dx[i]/k[i])*(T0_half[n]*(q_half[n][i]/T_half[n][i])**2)
                U_ex_st_cell[n][i] = dx[i]*C[i]*(1-(T0_half[n]/T_half[n][i]))*((T[n+1][i]-T[n][i])/t_step)
                U_ex_out_cell[n][i] = q_out_half[n][i]*(1-(T0_half[n]/T_R_half[n][i]))
                U_stored_ex_cell[n][i] = C[i]*dx[i]*((T[n][i]-T0[n])-T0[n]*math.log(T[n][i]/T0[n])) 

            

        q_tot = [K_tot(Wall)*(BC_L[n] - BC_R[n]) for n in range(t_row-1)]

        for n in range(t_row-1):
            D_ex_in[n]  = (1-(T0[n]/BC_L[n]))*q_tot[n]
            D_ex_c[n]   = R_tot(Wall)*(q_tot[n]**2/(BC_L[n]*BC_R[n]))*T0[n]
            D_ex_out[n] = (1-(T0[n]/BC_R[n]))*q_tot[n]
            
        for n in range(t_row):
            for i in range(N):
                T_L[n][i] = K2D(T_L[n][i])
                T[n][i]   = K2D(T[n][i])    
                T_R[n][i] = K2D(T_R[n][i])
        
        for n in range(t_row-1):
            for i in range(N):
                Carneff[n][i] = 1-T0[n]/T_half[n][i]

        DF_U_ex_c_cell = pd.DataFrame(U_ex_c_cell)
        DF_U_ex_c   = DF_U_ex_c_cell.sum(axis=1).iloc[resize_y0:resize_y1]
        DF_D_ex_c =   pd.DataFrame(D_ex_c).iloc[resize_y0:resize_y1]
        
        Total_U_ex_c = DF_U_ex_c.sum(axis=0)*t_step
        Total_D_ex_c = DF_D_ex_c.sum(axis=0)*t_step

        Error_rate = abs((Total_U_ex_c-Total_D_ex_c)/Total_U_ex_c)*100
        DF_Error_rate.iloc[alpha_idx,len_idx] = Error_rate

        # DF_T_L  = pd.DataFrame(T_L)
        # DF_T_R  = pd.DataFrame(T_R)
        # DF_T    = pd.DataFrame(T)

        # DF_q_in  = pd.DataFrame(q_in)
        # DF_q_out = pd.DataFrame(q_out)
        # DF_q    = pd.DataFrame(q)

        # DF_Carneff = pd.DataFrame(Carneff)

        # DF_U_ex_in_cell = pd.DataFrame(U_ex_in_cell)
        # DF_U_ex_c_cell = pd.DataFrame(U_ex_c_cell)
        # DF_U_ex_st_cell = pd.DataFrame(U_ex_st_cell)
        # DF_U_ex_out_cell = pd.DataFrame(U_ex_out_cell)
        # DF_U_stored_ex_cell = pd.DataFrame(U_stored_ex_cell)

        # DF_U_ex_in  = DF_U_ex_in_cell.sum(axis=1).to_frame()
        # DF_U_ex_c   = DF_U_ex_c_cell.sum(axis=1).to_frame()
        # DF_U_ex_st  = DF_U_ex_st_cell.sum(axis=1).to_frame()
        # DF_U_ex_out = DF_U_ex_out_cell.sum(axis=1).to_frame()

        # DF_D_ex_in =  pd.DataFrame(D_ex_in)
        # DF_D_ex_c =   pd.DataFrame(D_ex_c)
        # DF_D_ex_out = pd.DataFrame(D_ex_out)

        # file_path = "output"

        # DF_T_L.to_csv(f"{file_path}/{LNidx}_T_L.csv", index = False)
        # DF_T_R.to_csv(f"{file_path}/{LNidx}_T_R.csv", index = False)     
        # DF_T.to_csv(f"{file_path}/{LNidx}_T.csv", index = False)        
        
        # DF_q_in.to_csv(f"{file_path}/{LNidx}_q_in.csv", index = False) 
        # DF_q_out.to_csv(f"{file_path}/{LNidx}_q_out.csv", index = False)
        # DF_q.to_csv(f"{file_path}/{LNidx}_q.csv", index = False)  

        # DF_Carneff.to_csv(f"{file_path}/{LNidx}_Carneff.csv", index = False)

        # DF_U_ex_in_cell.to_csv(f"{file_path}/{LNidx}_U_ex_in_cell.csv", index = False)
        # DF_U_ex_c_cell.to_csv(f"{file_path}/{LNidx}_U_ex_c_cell.csv", index = False)
        # DF_U_ex_st_cell.to_csv(f"{file_path}/{LNidx}_U_ex_st_cell.csv", index = False)
        # DF_U_ex_out_cell.to_csv(f"{file_path}/{LNidx}_U_ex_out_cell.csv", index = False)
        # DF_U_stored_ex_cell.to_csv(f"{file_path}/{LNidx}_U_stored_ex_cell.csv", index = False)

        # DF_U_ex_in.to_csv(f"{file_path}/{LNidx}_U_ex_in.csv", index = False)
        # DF_U_ex_c.to_csv(f"{file_path}/{LNidx}_U_ex_c.csv", index = False)
        # DF_U_ex_st.to_csv(f"{file_path}/{LNidx}_U_ex_st.csv", index = False)
        # DF_U_ex_out.to_csv(f"{file_path}/{LNidx}_U_ex_out.csv", index = False)

        # DF_D_ex_in.to_csv(f"{file_path}/{LNidx}_D_ex_in.csv", index = False)
        # DF_D_ex_c.to_csv(f"{file_path}/{LNidx}_D_ex_c.csv", index = False)
        # DF_D_ex_out.to_csv(f"{file_path}/{LNidx}_D_ex_out.csv", index = False)
        print(f"[{alpha_idx+1}/{len(alpha)}][{len_idx+1}/{len(length)}]----------\nalpha = {alpha[alpha_idx]} [m2/s]\nlength = {length[len_idx]} [m]\nError rate = {Error_rate}")
DF_Error_rate.to_csv(f"{CaseName}", index=False)
print("END")